<a href="https://colab.research.google.com/github/gwendolynthompson/PoliticalProxyTuning/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gradio as gr
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.nn.functional import softmax

In [4]:
# Load models and tokenizer
base_model = GPT2LMHeadModel.from_pretrained("gpt2")
anti_expert_model = GPT2LMHeadModel.from_pretrained("distilgpt2")
expert_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab_Notebooks/Transformers/fine_tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Move models to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
anti_expert_model.to(device)
expert_model.to(device)

# Proxy-Tuning Function
def generate_answers(question):
    # Tokenize input
    inputs = tokenizer(question, return_tensors="pt").to(device)

    # Compute logits for all models
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        anti_expert_logits = anti_expert_model(**inputs).logits
        expert_logits = expert_model(**inputs).logits

    # Proxy-tuned logits
    alpha = 0.5  # Adjust as needed
    proxy_tuned_logits = base_logits + alpha * (expert_logits - anti_expert_logits)

    # Generate GPT-2 answer
    base_answer_ids = torch.argmax(softmax(base_logits, dim=-1), dim=-1)
    gpt2_answer = tokenizer.decode(base_answer_ids[0], skip_special_tokens=True)

    # Generate Proxy-Tuned answer
    proxy_tuned_answer_ids = torch.argmax(softmax(proxy_tuned_logits, dim=-1), dim=-1)
    proxy_tuned_answer = tokenizer.decode(proxy_tuned_answer_ids[0], skip_special_tokens=True)

    return gpt2_answer, proxy_tuned_answer

# Gradio Interface
def process_input(question):
    gpt2_answer, proxy_tuned_answer = generate_answers(question)
    return f"GPT-2 Answer: {gpt2_answer}\nProxy-Tuned Answer: {proxy_tuned_answer}"

# Set up Gradio app
interface = gr.Interface(
    fn=process_input,
    inputs="text",
    outputs="text",
    title="Proxy-Tuning Demonstration",
    description="Enter a question and see answers generated by GPT-2 (base model) and Proxy-Tuned logits."
)

# Launch Gradio interface
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9691f256362ee69004.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
